In [1]:
import torch
import numpy as np
import data
import tqdm
from torchvision.transforms import v2
import models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

/home/anirudh/Documents/GSoC/DeepLense/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
EPOCHS = 200
BATCH_SIZE = 64
LR = 1e-4
OUT_CLASSES = 3

In [3]:
def transform(image):
    """
    Data augmentation by random rotation, and random scaling between 0.8 to 1.2

    :param image: Image to be transformed
    :return: Transformed image
    """
    return v2.RandomAffine(degrees=(-180,180),scale=(0.8,1.2))(image)

model = models.Resnet_simple().to(device)

In [4]:
train_dataset = data.LensingDataset('index.csv','../dataset/train/',transform=transform)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [0.9, 0.1])

train_dataloader = torch.utils.data.DataLoader(train_dataset,shuffle=True,batch_size=BATCH_SIZE,num_workers=15)
val_dataloader = torch.utils.data.DataLoader(val_dataset,shuffle=True,batch_size=BATCH_SIZE,num_workers=15)

In [5]:
loss_fn = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(params=model.parameters(),lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt,'min',patience=2,factor=0.1)
history = {'val_loss':[], 'acc':[], 'loss':[]}
best_val_loss = np.inf
best_weights = None
for epoch in range(EPOCHS):
    losses = []
    accs = []
    model.train()
    for i,(x,y) in enumerate(tqdm.tqdm(train_dataloader, desc=f"Training epoch {epoch+1}")):
        (x, y) = (x.float().to(device), y.to(device))
        x = model(x)
        loss = loss_fn(x,y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        label_indices = torch.argmax(y, 1)
        prediction_indices = torch.argmax(x, 1)
        acc = float(torch.sum(label_indices == prediction_indices))/float(len(label_indices))
        losses.append(float(loss))
        accs.append(float(acc))
    print("Training at epoch %d/%d: %.4f with acc=%.5f at LR=%.6f"%(epoch+1,EPOCHS,np.mean(losses),np.mean(accs),opt.param_groups[0]['lr']))
    history['loss'].append(np.mean(losses))
    train_loss = np.mean(losses)
    model.eval()
    losses = []
    accs = []
    for i,(x,y) in enumerate(tqdm.tqdm(val_dataloader, desc=f"Validation epoch {epoch}")):
        (x, y) = (x.float().to(device), y.to(device))
        x = model(x)
        val_loss = loss_fn(x,y)
        label_indices = torch.argmax(y, 1)
        prediction_indices = torch.argmax(x, 1)
        acc = float(torch.sum(label_indices == prediction_indices))/float(len(label_indices))
        losses.append(float(val_loss))
        accs.append(float(acc))
    if best_val_loss > np.mean(losses):
        best_val_loss = np.mean(losses)
        best_weights = model.state_dict()
    history['val_loss'].append(np.mean(losses))
    history['acc'].append(np.mean(accs))
    print(f"Validation at epoch {epoch+1}/{EPOCHS}: {np.mean(losses)} with acc={np.mean(accs)} at LR={opt.param_groups[0]['lr']}")
    scheduler.step(train_loss)

Training epoch 1: 100%|██████████| 422/422 [00:42<00:00,  9.96it/s]


Training at epoch 1/200: 1.1314 with acc=0.33868 at LR=0.000100


Validation epoch 0: 100%|██████████| 47/47 [00:02<00:00, 20.12it/s]


Validation at epoch 1/200: 1.104361912037464 with acc=0.3464570668693009 at LR=0.0001


Training epoch 2: 100%|██████████| 422/422 [00:39<00:00, 10.78it/s]


...

Validation at epoch 199/200: 0.782251719464647 with acc=0.7624905015197568 at LR=1.0000000000000004e-08


Training epoch 200: 100%|██████████| 422/422 [00:39<00:00, 10.71it/s]


Training at epoch 200/200: 0.7181 with acc=0.82920 at LR=0.000000


Validation epoch 199: 100%|██████████| 47/47 [00:02<00:00, 20.15it/s]

Validation at epoch 200/200: 0.7702006710336563 with acc=0.7769281914893617 at LR=1.0000000000000004e-08


In [7]:
torch.save(model.state_dict(), '../trained_weights/resnet_simple.pt')